In [2]:
#Shengtao Lin, Tianhe Wang

In [1]:
!pip install tweepy

In [2]:
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream

In [3]:
#key and tokens
#api key: bUvI6HoYH97pwjTmnMNAPTMTS
#api secret key: Hp420vZADZ25NMR0sqEf4eVlYXVtUsyL4AhwgdAxTf2ssbr0in
#access token: 1190435999551303680-vK990ORPT4tj7KU1TTkfOtARxNzLfz
#Access Token Secret: 6KWkz1UYuCPvvGKk22AvMumRpwlA2kO8DlIVZ5KwpQtTK

access_token = "1190435999551303680-vK990ORPT4tj7KU1TTkfOtARxNzLfz"
access_token_secret = "6KWkz1UYuCPvvGKk22AvMumRpwlA2kO8DlIVZ5KwpQtTK"
consumer_key = "bUvI6HoYH97pwjTmnMNAPTMTS"
consumer_secret = "Hp420vZADZ25NMR0sqEf4eVlYXVtUsyL4AhwgdAxTf2ssbr0in"

In [16]:
import sys
default_stdout = sys.stdout
sys.stdout = open('AI-out', 'w')

In [17]:
class StdOutListener(StreamListener):

    def on_data(self, data):
        print (data)
        return True

    def on_error(self, status):
        print (status)


if __name__ == '__main__':

    #This handles Twitter authetification and the connection to Twitter Streaming API
    l = StdOutListener()
    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    stream = Stream(auth, l)

    #This line filter Twitter Streams to capture data by the keywords: 'python', 'javascript', 'ruby'
    stream.filter(track=['#ArtificialIntelligence'],languages=["en"])

KeyboardInterrupt: 

In [18]:
sys.stdout = default_stdout

In [19]:
import json
count=0
with open("test-out", "r") as f1:
    for line in f1:
        try:
            data = json.loads(line)
            # if you want to see a specific field, you can print it. 
            #If your file is big, there may be too many of these printed
            #print(data['text']) 
            count+=1
        except:
            continue

In [13]:
data

{'created_at': 'Sat Apr 10 05:38:55 +0000 2021',
 'id': 1380757162557894657,
 'id_str': '1380757162557894657',
 'text': 'RT @KevinClarity: “How I Calculated the 1,000,000th Fibonacci Number with Python” by Kush\nhttps://t.co/NrJmBHl8fw\n\n #Machinelearning #100Da…',
 'source': '<a href="https://www.polynique.com" rel="nofollow">Hugo123</a>',
 'truncated': False,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 1357677849101160449,
  'id_str': '1357677849101160449',
  'name': 'Hugo',
  'screen_name': 'Hugone21',
  'location': None,
  'url': None,
  'description': 'In love with #dev and #coding',
  'translator_type': 'none',
  'protected': False,
  'verified': False,
  'followers_count': 67,
  'friends_count': 6,
  'listed_count': 0,
  'favourites_count': 21793,
  'statuses_count': 3938,
  'created_at': 'Fri Feb 05 13:10:10 +0000 2021',
  'utc_offset': None

In [20]:
count

10